In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\prajw\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [2]:
url = 'https://store.unionlosangeles.com/collections/outerwear'  # outerwear page in union los angeles page
driver.get(url)

def infinite_scroll():
    # get current height of the page
    height = driver.execute_script('return document.body.scrollHeight')
    while True:
        # scroll to end of the page
        driver.execute_script(f'window.scrollTo(0, document.body.scrollHeight)')

        # wait for 4 seconds so that the items in the page get loaded
        time.sleep(4)

        # find out the new height of the page after scrolling
        new_height = driver.execute_script('return document.body.scrollHeight')

        if new_height == height:  # you have reached the end of the page, so break
            break

        # you have not yet reached the end of the page, so repeat untill you reach the end of the page
        height = new_height

# this function can be used anytime a page with infinite scrolling has to be scraped
infinite_scroll()

In [3]:
soup = BeautifulSoup(driver.page_source, parser='lxml')
blocks = soup.find_all('ul',class_='small-block-grid-2 medium-block-grid-4 large-block-grid-6 collection-grid')
# 0th block indicates most popular items.They will be repeated in all products.Hence skip to avoid duplicate products
df = pd.DataFrame(columns = ['Product Name','Designer Name','Price','Link'])
for block in blocks[1:]:
    product_blocks = block.find_all('li')
    for product in product_blocks:
        try:
            index = len(df)
            product_name = product.find('h3',class_='cap-title').text
            designer_name = product.find('h3',class_='cap-vendor').text
            price = product.find('h3',class_='cap-price').text
            link = 'https://store.unionlosangeles.com' + product.find('a').get('href')
            df.loc[index] = [product_name, designer_name, price, link]
        except:
            pass

df.to_csv('union los angeles data.csv', index=False)

In [4]:
len(df)

99

In [5]:
df.head()

,Product Name,Designer Name,Price,Link
0,Bal E-Coat,Neighborhood,$816,https://store.unionlosangeles.com/products/bal...
1,Tactical Mod / CN-Jacket,Neighborhood,$954,https://store.unionlosangeles.com/products/tac...
2,Smock Mod / CE-Jacket,Neighborhood,$858,https://store.unionlosangeles.com/products/smo...
3,Smock Mod / CE- Jacket,Neighborhood,$858,https://store.unionlosangeles.com/products/smo...
4,Climax at 29 Western Jacket,ENFANTS RICHES DÉPRIMÉS,"$1,950",https://store.unionlosangeles.com/products/cli...


- let's find out which product is the most expensive
- convert the each price in the price series to an integer after removing '$' and ',' signs
- find out the maximum in the series
- this represents the maximum price out of all products

In [6]:
df.Price.apply(lambda s:int(s[1:].replace(',',''))).max()

4647

- now find out the product that corresponds to the maximum price

In [7]:
df[df.Price == '$4,647']

,Product Name,Designer Name,Price,Link
98,Varsity SB JKT (Wool),Visvim,"$4,647",https://store.unionlosangeles.com/products/var...
